# Setup

In [1]:
# Install dependencies
from IPython.display import clear_output

!pip3 install gym
clear_output()

!rm -rf jelly-bean-world/ || true
!git clone https://github.com/NishanthVAnand/jelly-bean-world.git
clear_output()

%cd jelly-bean-world/
clear_output()

!rm -rf jbw/deps/core
!git rm jbw/deps/core
!git submodule add https://github.com/asaparov/core.git jbw/deps/core
clear_output()

!rm -rf jbw/deps/math
!git rm jbw/deps/math
!git submodule add https://github.com/asaparov/math.git jbw/deps/math
clear_output()

!git submodule update --init --recursive

%cd api/python/
!python setup.py install
clear_output()

%cd ../../..

import jbw
import gym

/content


In [2]:
# Mount drive and create shortcut

from google.colab import drive
import sys
import os
import shutil
import warnings

drive.mount('/content/gdrive')
folder = "/content/gdrive/MyDrive/COMP579/Final_Project/" #@param {type:"string"}
!ln -Ts $folder /content/project 2> /dev/null

# Add the assignment folder to Python path
if '/content/project' not in sys.path:
    sys.path.insert(0, '/content/project')

Mounted at /content/gdrive


In [3]:
%cd project/

/content/gdrive/MyDrive/COMP579/Final_Project


# Training Code

In [4]:
from dataclasses import dataclass
from train_agent import evaluate_agent, get_environment, train_agent
from os import listdir
from os.path import isfile, join
import importlib

@dataclass
class Arguments:
  group: str = 'GROUP_011'

args = Arguments()

In [ ]:
path = './'+args.group+'/'
files = [f for f in listdir(path) if isfile(join(path, f))]
if ('agent.py' not in files) or ('env_info.txt' not in files):
  print("Your GROUP folder does not contain agent.py or env_info.txt!")
  exit()

with open(path+'env_info.txt') as f:
  lines = f.readlines()
env_type = lines[0].lower()

env = get_environment(env_type) 
env_eval = get_environment(env_type)
if 'jellybean' in env_type:
  env_specs = {'scent_space': env.scent_space, 'vision_space': env.vision_space, 'feature_space': env.feature_space, 'action_space': env.action_space}
if 'mujoco' in env_type:
  env_specs = {'observation_space': env.observation_space, 'action_space': env.action_space}
agent_module = importlib.import_module(args.group+'.agent')
agent = agent_module.Agent(env_specs)

# Note these can be environment specific and you are free to experiment with what works best for you
total_timesteps = 2000000
evaluation_freq = 1000
n_episodes_to_evaluate = 20

learning_curve = train_agent(agent, env, env_eval, total_timesteps, evaluation_freq, n_episodes_to_evaluate)
